In [ ]:
import torch

%run '/content/drive/MyDrive/Colab Notebooks/CPSC 490/model/net_utils.ipynb'

Mounted at /content/drive


In [ ]:
class NeuralNetwork(torch.nn.Module):
    '''
    Neural network class of fully connected layers

    Arg(s):
        n_input_feature : int
            number of input features
        n_output : int
            number of output classes
    '''

    def __init__(self, n_input_feature, n_output):
        super(NeuralNetwork, self).__init__()

        # Create your 6-layer neural network using fully connected layers with ReLU activations
        # https://pytorch.org/docs/stable/generated/torch.nn.Linear.html
        # https://pytorch.org/docs/stable/generated/torch.nn.functional.relu.html
        # https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html

        # TODO: Instantiate 5 fully connected layers and choose number of neurons i.e. 512
        self.fully_connected_layer1 = torch.nn.Linear(in_features=n_input_feature, out_features=512)
        self.fully_connected_layer2 = torch.nn.Linear(in_features=512, out_features=256)
        self.fully_connected_layer3 = torch.nn.Linear(in_features=256, out_features=128)
        self.fully_connected_layer4 = torch.nn.Linear(in_features=128, out_features=64)
        self.fully_connected_layer5 = torch.nn.Linear(in_features=64, out_features=32)

        # TODO: Define output layer
        self.output = torch.nn.Linear(in_features=32, out_features=n_output)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        '''
        Forward pass through the neural network

        Arg(s):
            x : torch.Tensor[float32]
                tensor of N x d
        Returns:
            torch.Tensor[float32]
                tensor of n_output predicted class
        '''

        # TODO: Implement forward function
        output_fc1 = self.relu(self.fully_connected_layer1(x))
        output_fc2 = self.relu(self.fully_connected_layer2(output_fc1))
        output_fc3 = self.relu(self.fully_connected_layer3(output_fc2))
        output_fc4 = self.relu(self.fully_connected_layer4(output_fc3))
        output_fc5 = self.relu(self.fully_connected_layer5(output_fc4))

        output_logits = self.output(output_fc5)

        return output_logits

In [ ]:
class ResNet18Encoder(torch.nn.Module):
    '''
    ResNet18 encoder with skip connections

    Arg(s):
        input_channels : int
            number of channels in input data
        n_filters : list
            number of filters to use for each block
        weight_initializer : str
            kaiming_normal, kaiming_uniform, xavier_normal, xavier_uniform
        activation_func : func
            activation function after convolution
        use_batch_norm : bool
            if set, then applied batch normalization
        use_instance_norm : bool
            if set, then applied instance normalization
    '''

    def __init__(self,
                 input_channels=3,
                 n_filters=[64, 64, 128, 256, 512],
                 weight_initializer='kaiming_uniform',
                 activation_func='leaky_relu',
                 use_batch_norm=False,
                 use_instance_norm=False):
        super(ResNet18Encoder, self).__init__()

        assert len(n_filters) == 5

        n_blocks = [2, 2, 2, 2]
        resnet_block = ResNetBlock

        activation_finc = activation_function(activation_func)

        for n in range(len(n_filters) - len(n_blocks) - 1):
            n_blocks = n_blocks + [n_blocks[-1]]


        assert len(n_filters) == len(n_blocks) + 1

        # Keep track on current block
        block_idx = 0
        filter_idx = 0

        activation_func = activation_function(activation_func)

        # TODO: Implement ResNet encoder using ResNeBlock from net_utils.py
        # Based on https://arxiv.org/abs/1512.03385

        self.model = []
        x = Conv2d(in_channels=input_channels,
                  out_channels=n_filters[filter_idx],
                  kernel_size=7,
                  stride=2,
                  weight_initializer=weight_initializer,
                  activation_func=activation_func,
                  use_batch_norm=use_batch_norm,
                  use_instance_norm=use_instance_norm)
        self.model.append(x)
        x = torch.nn.MaxPool2d(kernel_size=3, stride=2)
        self.model.append(x)
        filter_idx += 1

        for layer in range(0, len(n_blocks)):
            for block in range(0, n_blocks[layer]):
                if block == 0 and layer != 0:
                    stride = 2
                else:
                    stride = 1
                if block == 0:
                  x = resnet_block(in_channels=n_filters[filter_idx-1],
                                    out_channels=n_filters[filter_idx-1],
                                    stride = stride,
                                    weight_initializer=weight_initializer,
                                    activation_func=activation_func,
                                    use_batch_norm=use_batch_norm,
                                    use_instance_norm=use_instance_norm)
                  self.model.append(x)
                else:
                  x = resnet_block(in_channels=n_filters[filter_idx-1],
                                    out_channels=n_filters[filter_idx],
                                    stride = stride,
                                    weight_initializer=weight_initializer,
                                    activation_func=activation_func,
                                    use_batch_norm=use_batch_norm,
                                    use_instance_norm=use_instance_norm)
                  self.model.append(x)
            filter_idx += 1
            block_idx += 1

        self.model = torch.nn.Sequential(*self.model)



    def forward(self, x):
        '''
        Forward input x through a ResNet encoder

        Arg(s):
            x : torch.Tensor[float32]
                N x C x H x W input tensor
        Returns:
            torch.Tensor[float32] : N x K x h x w output tensor
            list[torch.Tensor[float32]] : list of intermediate feature maps used for skip connections
        '''
        layers = [x]

        # TODO: Implement forward function

        layer_count = 0
        for conv in self.model:
            layer_count += 1

            layers.append(conv(layers[-1]))

        return layers[-1], layers[1:-2]

In [ ]:
class VGGNet11Encoder(torch.nn.Module):
    '''
    VGGNet encoder with skip connections

    Arg(s):
        input_channels : int
            number of channels in input data
        n_filters : list
            number of filters to use for each block
        weight_initializer : str
            kaiming_normal, kaiming_uniform, xavier_normal, xavier_uniform
        activation_func : func
            activation function after convolution
        use_batch_norm : bool
            if set, then applied batch normalization
        use_instance_norm : bool
            if set, then applied instance normalization
    '''

    def __init__(self,
                 input_channels=512,
                 n_filters=[32, 64, 128, 256, 256],
                 weight_initializer='kaiming_uniform',
                 activation_func='leaky_relu',
                 use_batch_norm=False,
                 use_instance_norm=False):
        super(VGGNet11Encoder, self).__init__()

        activation_func = activation_function(activation_func)

        # TODO: Implement VGGNet encoder using VGGNetBlock from net_utils.py
        # Based on https://arxiv.org/pdf/1409.1556.pdf

        n_convolutions = [1, 1, 2, 2, 2]  # VGG11
        stride = 1

        assert len(n_convolutions) == len(n_filters)
        block_num = 0

        in_channels, out_channels = [input_channels, n_filters[block_num]]

        self.conv1 = Conv2d(
            in_channels,
            out_channels,
            kernel_size=3,
            stride=stride,
            weight_initializer=weight_initializer,
            activation_func=activation_func,
            use_batch_norm=use_batch_norm,
            use_instance_norm=use_instance_norm
        )

        block_num = 1
        in_channels, out_channels = [n_filters[block_num - 1], n_filters[block_num]]

        self.conv2 = VGGNetBlock(
            in_channels,
            out_channels,
            n_convolution=n_convolutions[block_num],
            stride=2,
            weight_initializer=weight_initializer,
            activation_func=activation_func,
            use_batch_norm=use_batch_norm,
            use_instance_norm=use_instance_norm)

        block_num = 2
        in_channels, out_channels = [n_filters[block_num - 1], n_filters[block_num]]

        self.conv3 = VGGNetBlock(
            in_channels,
            out_channels,
            n_convolution=n_convolutions[block_num],
            stride=2,
            weight_initializer=weight_initializer,
            activation_func=activation_func,
            use_batch_norm=use_batch_norm,
            use_instance_norm=use_instance_norm)

        block_num = 3
        in_channels, out_channels = [n_filters[block_num - 1], n_filters[block_num]]

        self.conv4 = VGGNetBlock(
            in_channels,
            out_channels,
            n_convolution=n_convolutions[block_num],
            stride=2,
            weight_initializer=weight_initializer,
            activation_func=activation_func,
            use_batch_norm=use_batch_norm,
            use_instance_norm=use_instance_norm)

        block_num = 4
        in_channels, out_channels = [n_filters[block_num - 1], n_filters[block_num]]

        self.conv5 = VGGNetBlock(
            in_channels,
            out_channels,
            n_convolution=n_convolutions[block_num],
            stride=2,
            weight_initializer=weight_initializer,
            activation_func=activation_func,
            use_batch_norm=use_batch_norm,
            use_instance_norm=use_instance_norm)

        self.max_pool = torch.nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        '''
        Forward input x through a VGGNet encoder

        Arg(s):
            x : torch.Tensor[float32]
                N x C x H x W input tensor
        Returns:
            torch.Tensor[float32] : N x K x h x w output tensor
            list[torch.Tensor[float32]] : list of intermediate feature maps used for skip connections
        '''

        layers = [x]

        # TODO: Implement forward function
        layers.append(self.conv1(layers[-1]))
        layers.append(self.conv2(layers[-1]))
        layers.append(self.conv3(layers[-1]))
        layers.append(self.conv4(layers[-1]))
        layers.append(self.conv5(layers[-1]))
        layers.append(self.max_pool(layers[-1]))

        # Return latent and intermediate features
        return layers[-1], layers[1:-1]